In [28]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from cleaner import DataCleaner
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from tools import (
    logging,
    bins_generator,
    import_datasets,
    decode_clean_actors,
    decode_clean,

)
import eda_movies as eda

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [161]:
# link = "big_dataframe/actors.parquet"
link = "movies_datasets/name_basics.tsv"
test = import_datasets(link, 'pandas', sep="\t")
test

2023-10-26 18:03:33 INFO     Pandas loaded ! Importing name_basics...


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0072308,tt0053137,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0075213,tt0037382,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0054452,tt0057345,tt0049189,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0072562,tt0078723,tt0077975"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0083922,tt0050976,tt0050986,tt0069467"
...,...,...,...,...,...,...
12933862,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department","tt2455546,tt11657662,tt14069590"
12933863,nm9993716,Essias Loberg,\N,\N,NaN,\N
12933864,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
12933865,nm9993718,Aayush Nair,\N,\N,cinematographer,tt8736744


In [185]:
df = test.copy()

In [186]:
df.drop(["deathYear", "primaryProfession"], axis=1, inplace=True)

In [187]:
clean = DataCleaner()
df = clean.fix_values(df, "fix_n")

2023-10-26 18:10:36 INFO     Fixing N values...


In [188]:
df["knownForTitles"] = np.where(df["knownForTitles"] == 0, "Unknown", df["knownForTitles"])
df["knownForTitles"] = df["knownForTitles"].str.split(",")
# mf["person_role"] = mf["person_role"].apply(decode_clean_actors).str.split(",")


In [196]:
df["birthYear"] = df["birthYear"].astype("int64")

In [198]:
df.to_parquet("clean_datasets/person_parquet")

In [206]:
# link = "clean_datasets/movies_cleaned.parquet"
link_principals = "movies_datasets/title_principals.tsv"

# movies_cleaned = import_datasets(link, "parquet")
principals = import_datasets(link_principals, "pandas", sep="\t")

2023-10-26 18:17:36 INFO     Pandas loaded ! Importing title_principals...


In [212]:
df_prin = principals.copy()

In [ ]:
df_prin.drop(["job"], inplace=True, axis=1)


In [216]:
df_prin

,tconst,ordering,nconst,category,characters
0,tt0000001,1,nm1588970,self,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N
2,tt0000001,3,nm0374658,cinematographer,\N
3,tt0000002,1,nm0721526,director,\N
4,tt0000002,2,nm1335271,composer,\N
...,...,...,...,...,...
58672901,tt9916880,5,nm0584014,director,\N
58672902,tt9916880,6,nm0996406,director,\N
58672903,tt9916880,7,nm1482639,writer,\N
58672904,tt9916880,8,nm2586970,writer,\N


In [217]:
df_prin = clean.fix_values(df_prin, "fix_n")


2023-10-26 18:21:39 INFO     Fixing N values...


In [218]:

df_prin["characters"] = np.where(df_prin["characters"] == 0, "Unknown", df_prin["characters"])
# df_prin["characters"] = df_prin["characters"].str.split(",")
df_prin["characters"] = df_prin["characters"].apply(decode_clean_actors).str.split(",")


In [224]:
df_prin["category"].unique()

KeyError: 'category'

In [223]:
actors_list = ["self", "actor", "actress"]

df_prin = df_prin['characters'].isin(actors_list)

# actors = df_prin["category"].str.contains("act")
# directors = df_prin["category"].str.contains("dire")
# writors = df_prin["category"].str.contains("wri")
# cinemato = df_prin["category"].str.contains("cinem")

KeyboardInterrupt: 

In [30]:
# link = "big_dataframe/actors.parquet"
# df_og = import_datasets(
#     link,
#     types="parquet",
# )
# cleaning = DataCleaner()
# df = cleaning.fix_values(df_og, "fix_n")
# # df = cleaning.fix_values(df1, "fix_encode")

2023-10-26 16:51:09 INFO     Parquet loaded ! Importing actors...
2023-10-26 16:51:10 INFO     Fixing N values...


In [31]:
df

,person_id,person_name,person_birthdate,person_role,person_index,person_film
0,nm0000001,Fred Astaire,1899,"[""Guy Holden""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
1,nm0000001,Fred Astaire,1899,"[""Huck Haines""]",2,"tt0050419,tt0072308,tt0053137,tt0031983"
2,nm0000001,Fred Astaire,1899,"[""Jerry Travers""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
3,nm0000001,Fred Astaire,1899,"[""Bake Baker""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
4,nm0000001,Fred Astaire,1899,"[""Lucky Garnett""]",1,"tt0050419,tt0072308,tt0053137,tt0031983"
...,...,...,...,...,...,...
1183372,nm9993616,Ryan Mac Lennan,0,"[""Mick Owen""]",4,tt4844148
1183373,nm9993650,Marcin Balcerak,0,"[""Guardian""]",4,tt8739208
1183374,nm9993693,Apsara Rani,1996,0,4,"tt12856788,tt8737752,tt8302382,tt13847502"
1183375,nm9993693,Apsara Rani,1996,"[""Vidhi""]",2,"tt12856788,tt8737752,tt8302382,tt13847502"


In [32]:
df["person_birthdate"] = df["person_birthdate"].astype("int64")

In [33]:
df.isna().sum()

person_id           0
person_name         0
person_birthdate    0
person_role         0
person_index        0
person_film         0
dtype: int64

In [34]:
df.dtypes

person_id           object
person_name         object
person_birthdate     int64
person_role         object
person_index         int64
person_film         object
dtype: object

In [35]:
df["person_film"] = df["person_film"].str.split(",")

In [36]:
df["person_role"] = np.where(df["person_role"] == 0, "Unknown", df["person_role"])

In [37]:
df.isna().sum()

person_id             0
person_name           0
person_birthdate      0
person_role           0
person_index          0
person_film         739
dtype: int64

In [38]:
df["person_film"] = np.where(df["person_film"].isna(), "Unknown", df["person_film"])

In [39]:
mf = df.copy()

In [40]:
mf["person_role"] = mf["person_role"].apply(decode_clean_actors).str.split(",")

In [41]:
mf.head()

,person_id,person_name,person_birthdate,person_role,person_index,person_film
0,nm0000001,Fred Astaire,1899,[Guy Holden],1,"[tt0050419, tt0072308, tt0053137, tt0031983]"
1,nm0000001,Fred Astaire,1899,[Huck Haines],2,"[tt0050419, tt0072308, tt0053137, tt0031983]"
2,nm0000001,Fred Astaire,1899,[Jerry Travers],1,"[tt0050419, tt0072308, tt0053137, tt0031983]"
3,nm0000001,Fred Astaire,1899,[Bake Baker],1,"[tt0050419, tt0072308, tt0053137, tt0031983]"
4,nm0000001,Fred Astaire,1899,[Lucky Garnett],1,"[tt0050419, tt0072308, tt0053137, tt0031983]"


In [ ]:
mf[mf["person_name"].str.contains('Eddie Murphy')]

In [43]:
mf.shape

(1183377, 6)

In [44]:
# merge avec movies

In [45]:
link = "clean_datasets/movies_cleaned.parquet"
link_principals = "movies_datasets/title_principals.tsv"

movies_cleaned = import_datasets(link, "parquet")
principals = import_datasets(link_principals, "pandas", sep="\t")

2023-10-26 16:51:20 INFO     Parquet loaded ! Importing movies_cleaned...
2023-10-26 16:51:20 INFO     Pandas loaded ! Importing title_principals...


In [46]:
principals = principals[["tconst", "nconst"]]

principals.head()

,tconst,nconst
0,tt0000001,nm1588970
1,tt0000001,nm0005690
2,tt0000001,nm0374658
3,tt0000002,nm0721526
4,tt0000002,nm1335271


In [122]:
movies_with_personid = pd.merge(
    movies_cleaned,
    principals,
    left_on = "titre_id",
    right_on = "tconst",
)
#  ICi on récuperer tous les acteurs avec leurs titres

In [123]:
condi2 = (
    (movies_with_personid["titre_id"].str.contains("tt0068991")) &
    (movies_with_personid["nconst"].str.contains("nm0000367"))
)
movies_with_personid[condi2]

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,titleId,region,cuts,tconst,nconst
58260,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,[FR],0.00,[fr],Released,tt0068991,FR,1970-1979,tt0068991,nm0000367


In [124]:
movies_with_personid["titre_id"].unique().size

36228

In [134]:
movies_with_personid

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,titleId,region,cuts,tconst,nconst
0,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0941700
1,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0409869
2,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0562920
3,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0103722
4,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0751245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352799,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,"[AR, ES, FR]",0.00,"[eu, es]",Released,tt9916362,FR,>2020,tt9916362,nm1893148
352800,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,"[AR, ES, FR]",0.00,"[eu, es]",Released,tt9916362,FR,>2020,tt9916362,nm3471432
352801,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,"[AR, ES, FR]",0.00,"[eu, es]",Released,tt9916362,FR,>2020,tt9916362,nm2970042
352802,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,"[AR, ES, FR]",0.00,"[eu, es]",Released,tt9916362,FR,>2020,tt9916362,nm4065853


In [153]:
actors_merged = pd.merge(
    mf,
    movies_with_personid,
    left_on = "person_id",
    right_on = "nconst",
    how="left"
)

gerard = movies_with_personid[movies_with_personid.nconst.isin(mf.person_id)]
# gerard.head()
# actors_merged = actors_merged.drop("nconst", axis=1)

In [157]:
gerard

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,titleId,region,cuts,tconst,nconst
1,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0409869
2,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0562920
3,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0103722
4,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0751245
8,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,[US],0.00,"[de, en]",Released,tt0062989,FR,1960-1969,tt0062989,nm0528121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352789,tt15398694,Convergence: Courage in a Crisis,2021,113,[Documentary],6.00,300,en,Convergence: Courage in a Crisis,4.67,"[GB, US]",0.00,"[en, hi, zh, es, pt, fa, fr]",Released,tt15398694,FR,>2020,tt15398694,nm3915542
352795,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,"[AR, ES, FR]",0.00,"[eu, es]",Released,tt9916362,FR,>2020,tt9916362,nm3766704
352796,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,"[AR, ES, FR]",0.00,"[eu, es]",Released,tt9916362,FR,>2020,tt9916362,nm0107165
352797,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,"[AR, ES, FR]",0.00,"[eu, es]",Released,tt9916362,FR,>2020,tt9916362,nm0266723


In [158]:
actors_merged = pd.merge(
    gerard,
    mf,
    left_on = "nconst",
    right_on = "person_id",
)

In [159]:
actors_merged

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,...,region,cuts,tconst,nconst,person_id,person_name,person_birthdate,person_role,person_index,person_film
0,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,FR,1960-1969,tt0062989,nm0409869,nm0409869,John Ireland,1914,[Pfc. Windy Craven],4,"[tt0040724, tt0054331, tt0039748, tt0041113]"
1,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,FR,1960-1969,tt0062989,nm0409869,nm0409869,John Ireland,1914,[Duke Martin],1,"[tt0040724, tt0054331, tt0039748, tt0041113]"
2,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,FR,1960-1969,tt0062989,nm0409869,nm0409869,John Ireland,1914,[Paul Lester],1,"[tt0040724, tt0054331, tt0039748, tt0041113]"
3,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,FR,1960-1969,tt0062989,nm0409869,nm0409869,John Ireland,1914,[Fantail],4,"[tt0040724, tt0054331, tt0039748, tt0041113]"
4,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,FR,1960-1969,tt0062989,nm0409869,nm0409869,John Ireland,1914,[Jack Burden],2,"[tt0040724, tt0054331, tt0039748, tt0041113]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116184,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,...,FR,>2020,tt9916362,nm3766704,nm3766704,Amaia Aberasturi,1997,[Leire],1,"[tt1583953, tt4944622, tt13457822, tt9916362]"
3116185,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,...,FR,>2020,tt9916362,nm3766704,nm3766704,Amaia Aberasturi,1997,[Unknown],1,"[tt1583953, tt4944622, tt13457822, tt9916362]"
3116186,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,...,FR,>2020,tt9916362,nm3766704,nm3766704,Amaia Aberasturi,1997,[Begoña],1,"[tt1583953, tt4944622, tt13457822, tt9916362]"
3116187,tt9916362,Coven,2020,92,"[Drama, History]",6.40,5571,eu,Akelarre,8.63,...,FR,>2020,tt9916362,nm3766704,nm3766704,Amaia Aberasturi,1997,[Ana],1,"[tt1583953, tt4944622, tt13457822, tt9916362]"


In [145]:
actors_merged["titre_id"].unique().size

35960

In [160]:
condi2 = (
    (actors_merged["titre_id"].str.contains("tt0068991")) &
    (actors_merged["nconst"].str.contains("nm0000367"))
)
actors_merged[condi2]

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,...,region,cuts,tconst,nconst,person_id,person_name,person_birthdate,person_role,person_index,person_film
1156006,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Salesman],3,"[tt0103594, tt0099699, tt0099334, tt0120744]"
1156007,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Lhomme de la plage],3,"[tt0103594, tt0099699, tt0099334, tt0120744]"
1156008,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Pierre Vaucher],2,"[tt0103594, tt0099699, tt0099334, tt0120744]"
1156009,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Jean-Claude],1,"[tt0103594, tt0099699, tt0099334, tt0120744]"
1156010,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Jean Lavallee],4,"[tt0103594, tt0099699, tt0099334, tt0120744]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156152,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Saintonge],4,"[tt0103594, tt0099699, tt0099334, tt0120744]"
1156153,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Gabriel Carvin],1,"[tt0103594, tt0099699, tt0099334, tt0120744]"
1156154,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Patrick],3,"[tt0103594, tt0099699, tt0099334, tt0120744]"
1156155,tt0068991,Nathalie Granger,1972,83,[Drama],6.70,654,fr,Nathalie Granger,1.83,...,FR,1970-1979,tt0068991,nm0000367,nm0000367,Gérard Depardieu,1948,[Raoul Taupin],1,"[tt0103594, tt0099699, tt0099334, tt0120744]"


In [82]:
df = actors_merged.copy()

In [83]:
df.head()

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,...,status,titleId,region,cuts,person_id,person_name,person_birthdate,person_role,person_index,person_film
0,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Pfc. Windy Craven],4,"[tt0040724, tt0054331, tt0039748, tt0041113]"
1,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Duke Martin],1,"[tt0040724, tt0054331, tt0039748, tt0041113]"
2,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Paul Lester],1,"[tt0040724, tt0054331, tt0039748, tt0041113]"
3,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Fantail],4,"[tt0040724, tt0054331, tt0039748, tt0041113]"
4,tt0062989,Fort Utah,1967,84,[Western],5.00,303,en,Fort Utah,3.05,...,Released,tt0062989,FR,1960-1969,nm0409869,John Ireland,1914,[Jack Burden],2,"[tt0040724, tt0054331, tt0039748, tt0041113]"


In [ ]:
condi2 = (
    (principals["tconst"].str.contains("tt0068991")) &
    (principals["nconst"].str.contains("nm0000367"))
)


In [84]:
len(df[df["person_name"].str.contains("Gérard Depardieu")])

18724

In [72]:
df.isna().sum()

titre_id                0
titre_str               0
titre_date_sortie       0
titre_duree             0
titre_genres            0
rating_avg              0
rating_votes            0
original_language       0
original_title          0
popularity              0
production_countries    0
revenue                 0
spoken_languages        0
status                  0
titleId                 0
region                  0
cuts                    0
person_id               0
person_name             0
person_birthdate        0
person_role             0
person_index            0
person_film             0
dtype: int64